# Индивидуальный проект. (Кузнецов Александр Юрьевич)

# 1.Описание набора данных и признаков.

Данный проект кредитного скоринга решает задачу предсказания вероятности успешного возврата кредита для каждого заёмщика.

В анализе будут участвовать два набора данных длиной порядка 100-150 тысяч строк: исторические данные (на которых будет обучаться модель) и эталонные данные (на которых будет применена модель). 

Ссылка на данные https://github.com/alexander15555/alexander155
    
Название переменной - описание (тип переменной):

1.SeriousDlqin2yrs - наблюдалась просрочка в размере 90 и более дней (Категориальная) - ЦЕЛЕВАЯ ПЕРЕМЕННАЯ

2.RevolvingUtilizationOfUnsecuredLines - общий баланс по кредитным картам и личным кредитам (Процентный)

3.Age - возраст заёмщика (целочисленный)

4.NumberOfTime30-59DaysPastDueNotWorse - количество просрочек за последние 2 года от 30 до 59 дней (Целочисленный)

5.DebtRatio - ежемесячные платежи по долгам, деленные на валовой доход в месяц (Процентный)

6.MonthlyIncome - ежемесячный доход (Целочисленный)

7.NumberOfOpenCreditLinesAndLoans	- количество открытых кредитов и кредитных карт (Целочисленный)

8.NumberOfTimes90DaysLate	- количество просрочек от 90 дней и более (Целочисленный)

9.NumberRealEstateLoansOrLines - количество ипотечных кредитов и кредитов на недвижимость (Целочисленный)

10.NumberOfTime60-89DaysPastDueNotWorse - количество просрочек за последние 2 года от 60 до 89 дней (Целочисленный)

11.NumberOfDependents - количество иждивенцев в семье, исключая себя (Целочисленный) 


С целью построения модели кредитного скоринга определяется зависимая переменная категориального типа. Это «SeriousDlqin2yrs» –  признак просроченной задолженности на 90 и более дней. В качестве независимых переменных предстают все остальные атрибуты.



# 2.Первичный анализ данных.

In [ ]:
import warnings
warnings.simplefilter('ignore')

import seaborn as sns
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import math
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
import pylab as pl
from matplotlib import pyplot
%matplotlib inline

In [ ]:
train = pd.read_csv('creditscore-train.csv', sep=',', encoding='cp1251')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

Пустые значения были обнаружены по двум атрибутам: MonthlyIncome «ежемесячный доход» и NumberOfDependents «количество иждивенцев».

In [ ]:
train['MonthlyIncome'].fillna(train['MonthlyIncome'].mean(), inplace = True)
train['NumberOfDependents'].fillna(0, inplace = True)

Заменим пропущенные значения по доходу на среднее по данному признаку. 
Заменим пропущенные значения по количеству иждивенцев на нулевое значение, поскольку более вероятно, что клиент не заполнил данное поле по причине отсутствия семьи.

In [ ]:
train['age'].max()

Видим что в данных имеются "выбросы" - некорректные значения сильно отклоняющиеся от средних величин.

In [ ]:
train = train.drop('Unnamed: 0', axis = 1)
train = train.rename(columns = {'NumberOfTime30-59DaysPastDueNotWorse' : 'NumberOfTime3059DaysPastDueNotWorse'})
train = train.rename(columns = {'NumberOfTime60-89DaysPastDueNotWorse': 'NumberOfTime6089DaysPastDueNotWorse'})
train1 = train['SeriousDlqin2yrs']
train1 = pd.DataFrame(train1)
train2 = train.drop('SeriousDlqin2yrs', axis = 1)

In [ ]:
RevolvingUtilizationOfUnsecuredLines = train2['RevolvingUtilizationOfUnsecuredLines']
age = train2['age']
NumberOfTime3059DaysPastDueNotWorse = train2['NumberOfTime3059DaysPastDueNotWorse']
DebtRatio = train2['DebtRatio']
MonthlyIncome = train2['MonthlyIncome']
NumberOfOpenCreditLinesAndLoans = train2['NumberOfOpenCreditLinesAndLoans']
NumberOfTimes90DaysLate = train2['NumberOfTimes90DaysLate']
NumberRealEstateLoansOrLines = train2['NumberRealEstateLoansOrLines']
NumberOfTime6089DaysPastDueNotWorse = train2['NumberOfTime6089DaysPastDueNotWorse']
NumberOfDependents = train2['NumberOfDependents']

In [ ]:
a = [RevolvingUtilizationOfUnsecuredLines, age, NumberOfTime3059DaysPastDueNotWorse, DebtRatio, MonthlyIncome, NumberOfOpenCreditLinesAndLoans, NumberOfTimes90DaysLate, NumberRealEstateLoansOrLines, NumberOfTime6089DaysPastDueNotWorse, NumberOfDependents]

В случае с «выбросами» использовалось правило трёх сигм. Следуя ему, вероятность того, что случайная величина отклонится от своего математического ожидания на большую величину, чем утроенное среднее квадратичное отклонение, практически равна нулю. 

In [ ]:
def outlier(col):
    col[(col < (col.mean() - 3 * col.std())) | (col > (col.mean() + 3 * col.std()))] = col.mean()
    return col  

In [ ]:
b = []
for par in a:
    b.append(outlier(par))

In [ ]:
data = pd.DataFrame(b)
data = data.T

# 3 и 4.Первичный визуальный анализ данных и Инсайты, найденные зависимости.

Значения по каждому атрибуту исследуемого дата-сета, которые выходят за рамки, установленные вышеупомянутым правилом, были заменены на среднее по своему признаку. Ниже визуальное представление данных.

In [ ]:
plot1 = data['RevolvingUtilizationOfUnsecuredLines'].hist(color = 'lightblue')
plot1.set_title("Revolving Utilization Of Unsecured Lines")
pl.show()

In [ ]:
plot1 = data['age'].hist(color = 'lightblue')
plot1.set_title("AGE")
pl.show()

In [ ]:
plot1 = data['NumberOfTime3059DaysPastDueNotWorse'].hist(color = 'lightblue')
plot1.set_title("Number Of Time 30-59 Days Past Due Not Worse")
pl.show()

In [ ]:
plot1 = data['DebtRatio'].hist(color = 'lightblue')
plot1.set_title("Debt Ratio")
pl.show()

In [ ]:
plot1 = data['MonthlyIncome'].hist(color = 'lightblue')
plot1.set_title("Monthly Income")
pl.show()

In [ ]:
plot1 = data['NumberOfOpenCreditLinesAndLoans'].hist(color = 'lightblue')
plot1.set_title("Number Of Open Credit Lines And Loans")
pl.show()

In [ ]:
plot1 = data['NumberOfTimes90DaysLate'].hist(color = 'lightblue')
plot1.set_title("Number Of Times 90 Days Late")
pl.show()

In [ ]:
plot1 = data['NumberRealEstateLoansOrLines'].hist(color = 'lightblue')
plot1.set_title("Number Real Estate Loans Or Lines")
pl.show()

In [ ]:
plot1 = data['NumberOfTime6089DaysPastDueNotWorse'].hist(color = 'lightblue')
plot1.set_title("Number Of Time 60-89 Days Past Due Not Worse")
pl.show()

In [ ]:
plot1 = data['NumberOfDependents'].hist(color = 'lightblue')
plot1.set_title("Number Of Dependents")
pl.show()

Резкие "выбросы" в данных не наблюдаются.Средний возраст заемщиков около 55 лет.

In [ ]:
cols = ['RevolvingUtilizationOfUnsecuredLines', 'age',
       'NumberOfTime3059DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime6089DaysPastDueNotWorse',
       'NumberOfDependents']
a4_dims = (12, 8)
df = data[cols].corr()
fig, ax = pyplot.subplots(figsize=a4_dims)
sns.heatmap(ax=ax, data=df)

Наблюдается корреляция между признаками "NumberRealEstateLoansOrLines" (Количество ипотечных кредитов и кредитов на недвижимость, включая кредитные линии собственного капитала) и "NumberOfOpenCreditLinesAndLoans" (Количество открытых кредитов и кредитных карт).

Также наблюдаетcя более слабая корреляция между признаками:
1. "NumberOfTimes90DaysLate" (количество просрочек от 90 дней и более) и "NumberOfTime6089DaysPastDueNotWorse" (количество просрочек за последние 2 года от 60 до 89 дней).
2. "NumberOfTime6089DaysPastDueNotWorse" (количество просрочек за последние 2 года от 60 до 89 дней) и "NumberOfTime3059DaysPastDueNotWorse" (Количество просрочек за последние 2 года от 30 до 59 дней)
3. "NumberRealEstateLoansOrLines" (Количество ипотечных кредитов и кредитов на недвижимость, включая кредитные линии собственного капитала) и "MonthlyIncome" (Ежемесячный доход).

In [ ]:
sns.pairplot(data[['NumberOfOpenCreditLinesAndLoans','NumberRealEstateLoansOrLines',
                   'NumberOfTime6089DaysPastDueNotWorse','NumberOfTimes90DaysLate',
                   'NumberOfTime3059DaysPastDueNotWorse','MonthlyIncome']])

# 5. Выбор метрики

В качестве основной метрики для оценки качества модели будет выбран критерий ROC-AUC. Метрика ROC AUC зависит не от строгой дискретной классификации,а от предсказанных вероятностей классов, поэтому она более применима к задаче такого типа. При этом она подходит для несбалансированных данных.

ROC-кривая – это графическая характеристика качества бинарного классификатора, отражает зависимость количества верно классифицированных положительных примеров от количества неверно классифицированных отрицательных примеров. AUC является количественным показателем ROC: это площадь, ограниченная осями и ROC-кривой.



# 6. Выбор модели

Выбор модели - логистическая регрессия. 

Простой и быстрый вариант.Логистическая регрессия выдает вероятности отнесения к разным классам (это очень ценится в кредитном скоринге). Также логистическая регрессия хорошо интерпретируема и используется в реальном банковском моделировании рисков. 

# 7. Предобработка данных

Предобработка данных для данной работы была проведена в пунктах ранее.

Первичная обработка данных представляет из себя замену пустых значений и избавление от «выбросов». Пустые значения на данных для обучения были обнаружены по двум атрибутам: «ежемесячный доход» и «количество иждивенцев». Было принято решение заменить пропущенные значения по доходу на среднее по данному признаку. А по количеству иждивенцев – на нулевое значение, поскольку более вероятно, что клиент не заполнил данное поле по причине отсутствия семьи. В случае с «выбросами» использовалось правило трёх сигм. Следуя ему, вероятность того, что случайная величина отклонится от своего математического ожидания на большую величину, чем утроенное среднее квадратичное отклонение, практически равна нулю.

# 8,9,10. Кросс-валидация и настройка гиперпараметров модели, создание новых признаков и описание этого процесса, построение кривых валидации и обучения

In [ ]:
X = data
Y = train1

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
kf = KFold(n=150000, n_folds=5, shuffle=True)

Используемые для формирования модели информационные данные являются исторической выборкой, которая наиболее точно отображает генеральную совокупность исследуемых людей. Её разбиение проводилось при помощи техники кросс-валидации по 5 блокам с использованием генератора разбиений KFold.

Предобработка данных и признаков производилась в пункта ранее.

In [ ]:
def my_scorer(estimator, testX, testY):
    predicted_testY = estimator.predict_proba(testX)[:, 1]
    return roc_auc_score(testY, predicted_testY)

In [ ]:
scaler = StandardScaler()
scaledX = scaler.fit_transform(X)

Перед разбиением производилось случайное перемешивание объектов выборки, чтобы избежать смещённости кросс-валидационной оценки. Реализация логистической регрессии показала, что наибольшая AUC-ROC величина наблюдалась при значении свободного члена С = 0,0001. 

In [ ]:
def LogR_teach(C_value):
    clf = LogisticRegression(penalty='l2', C=C_value)
    return cross_val_score(clf, scaledX, Y, cv=kf, scoring=my_scorer).mean()

In [ ]:
def C_variants():
    for power in range(-4, 2):
        C = math.pow(10, power)
        quality = LogR_teach(C)
        print ('C=', C, ', quality=', quality)

In [ ]:
C_variants()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(scaledX, Y, test_size=.2, random_state=0)
clf = LogisticRegression(penalty='l2', C=0.0001)
clf.fit(X_train, y_train)
y_score = clf.predict_proba(X_test)[:, 1]

In [ ]:
plt.figure()
line_width = 1
fpr, tpr, thresholds = roc_curve(y_test, y_score)
plt.plot(fpr, tpr, color='navy', lw=line_width, label='LogRegression, C=0.0001')
plt.plot([0, 1], [0, 1], color='lightblue', lw=line_width, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()

Показателем качества модели является величина площади под ROC-кривой 0,817.

# 11. Прогноз для тестовой или отложенной выборки

Приведем отложенный датасет к надлежащему виду, повторив все преобразования. Затем применим логистическую регрессию и вычислим предсказания на тестовом дата-сете.

In [ ]:
test = pd.read_csv('creditscore-test.csv', sep=',', encoding='cp1251')

In [ ]:
test.info()

In [ ]:
test.isnull().sum()

Пустые значения были обнаружены по двум атрибутам: MonthlyIncome «ежемесячный доход» и NumberOfDependents «количество иждивенцев».

In [ ]:
test['MonthlyIncome'].fillna(test['MonthlyIncome'].mean(), inplace = True)
test['NumberOfDependents'].fillna(0, inplace = True)

Заменим пропущенные значения по доходу на среднее по данному признаку. Заменим пропущенные значения по количеству иждивенцев на нулевое значение, поскольку более вероятно, что клиент не заполнил данное поле по причине отсутствия семьи.

In [ ]:
del test['Unnamed: 0']
del test['SeriousDlqin2yrs']

In [ ]:
test = test.rename(columns = {'NumberOfTime30-59DaysPastDueNotWorse' : 'NumberOfTime3059DaysPastDueNotWorse'})
test = test.rename(columns = {'NumberOfTime60-89DaysPastDueNotWorse': 'NumberOfTime6089DaysPastDueNotWorse'})

In [ ]:
RevolvingUtilizationOfUnsecuredLines = test['RevolvingUtilizationOfUnsecuredLines']
age = test['age']
NumberOfTime3059DaysPastDueNotWorse = test['NumberOfTime3059DaysPastDueNotWorse']
DebtRatio = test['DebtRatio']
MonthlyIncome = test['MonthlyIncome']
NumberOfOpenCreditLinesAndLoans = test['NumberOfOpenCreditLinesAndLoans']
NumberOfTimes90DaysLate = test['NumberOfTimes90DaysLate']
NumberRealEstateLoansOrLines = test['NumberRealEstateLoansOrLines']
NumberOfTime6089DaysPastDueNotWorse = test['NumberOfTime6089DaysPastDueNotWorse']
NumberOfDependents = test['NumberOfDependents']

In [ ]:
c = [RevolvingUtilizationOfUnsecuredLines, age, NumberOfTime3059DaysPastDueNotWorse, DebtRatio, MonthlyIncome, NumberOfOpenCreditLinesAndLoans, NumberOfTimes90DaysLate, NumberRealEstateLoansOrLines, NumberOfTime6089DaysPastDueNotWorse, NumberOfDependents]

In [ ]:
d = []
for par in c:
    d.append(outlier(par))

In [ ]:
test_ = pd.DataFrame(d)
test_ = test_.T

In [ ]:
test.head()

In [ ]:
scaled_test = scaler.transform(test_)

In [ ]:
clf = LogisticRegression(penalty='l2', C=0.0001)
clf.fit(scaledX, Y)

Применяется модель логистической регрессии с наиболее подходящим коэффициентом С = 0,0001.

In [ ]:
prediction = clf.predict_proba(scaled_test)[:, 1]

In [ ]:
min(prediction), max(prediction)

In [ ]:
result = pd.DataFrame(np.array(prediction), columns=['probability'], index=test_.index)

In [ ]:
result[result['probability']>0.5].count()

In [ ]:
result.count()

Тоесть доля заёмщиков, у которых предсказанная вероятность больше 0,5 составляет чуть более 1,5 % от общего количества людей в тестовой выборке. 

In [ ]:
result

# 12. Выводы

Была построена модель для кредитного скоринга, которая решает задачу предсказания вероятности успешного возврата кредита для каждого заёмщика.

На исторических данных полученное качество модели, оцененное с поомщью метрики ROC-AUС составляет 0.817, что говорит о хорошей предсказательной способности модели.

На тестовой выборке были получены предсказанные вероятности, определяющие кредитоспособность клиентов. Доля заёмщиков, у которых предсказанная вероятность больше 0,5 составляет чуть более 1,5 % от общего количества людей в тестовой выборке.

Улучшить модель возможно, если:
1.более точно отбрать важные для модели признаки
2.использовать дополнительно внешние данные
3.определить ошибки алгоритма и скорректировать модель.